In [1]:
import schedule
import time
import pandas as pd
from datetime import datetime, timedelta, date
import pyodbc
from sqlalchemy import create_engine
import mysql.connector
import csv

In [2]:
db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '8november2002',
    'database': 'bhavDB',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

In [3]:
file=pd.read_csv(f"/Users/vinayak/Desktop/IIT Roorkee/SPY/Finetuning Model/greenSPY.csv")
for index, row in file.iterrows():
    symbol = row['SYMBOL']
    date = row['DATE']
    stop_loss=row['STOP_LOSS']
    date_object = datetime.strptime(date, "%d-%b-%Y")
    date = date_object.date()
    query=f"SELECT * FROM StockData2024 WHERE SYMBOL='{symbol}' AND  DATE1 ='{date}'"
    cursor.execute(query)
    result=cursor.fetchall()
    
    column_names = [desc[0] for desc in cursor.description]

    data= pd.DataFrame(result, columns=column_names)

    if not data.empty:
        entry_price = data['CLOSE_PRICE'].iloc[0]
        target_price = 1.20 * entry_price
    

        # print(symbol, entry_price, target_price, stop_loss)
        current_date=date
        day=0
        while(True):
            # date+1
            current_date+=timedelta(days=1)
            # get data for date and check if data is not empty
            query=f"SELECT * FROM StockData2024 WHERE SYMBOL='{symbol}' AND  DATE1 ='{current_date}'"
            cursor.execute(query)
            result=cursor.fetchall()
            day+=1
            if(day==300):
                break
            if(len(result)!=0):
                # check target and stoploss
                data= pd.DataFrame(result, columns=column_names)
                if(data.empty):
                    continue
                data=data.iloc[0]
                if(data['HIGH_PRICE']>=target_price).any():
                    filename = 'backtestingGREEN2.csv'
                    with open(filename, 'a', newline='') as csvfile:
                        csvwriter = csv.writer(csvfile)
                        p_l=target_price-entry_price
                        roi=100*p_l/entry_price
                        csvwriter.writerows([[date,symbol,entry_price,"Buy",target_price,stop_loss,p_l,roi,target_price,"Target Reached",current_date]])
                    break
                elif(data['CLOSE_PRICE']<=stop_loss).any():
                    filename = 'backtestingGREEN2.csv'
                    with open(filename, 'a', newline='') as csvfile:
                        csvwriter = csv.writer(csvfile)
                        p_l=data['CLOSE_PRICE']-entry_price
                        roi=100*p_l/entry_price
                        csvwriter.writerows([[date,symbol,entry_price,"Buy",target_price,stop_loss,p_l,roi,target_price,"Stoploss Hit",current_date]])
                    break
            # EntryDate,Symbol,Entry_Price,Buy/Sell,Target,Stoploss,P_L,ROI,Exit_Price,Status,Exit_Date


In [4]:
connection.commit()
cursor.close()
connection.close()